In [ ]:
import numpy as np
rng = np.random.default_rng(1)

In [ ]:
def KL(p, q): # compute Kullback-Leibler divergence (d in paper). check edge cases.
    if (p == 0 and q == 0) or (p == 1 and q == 1) or p == 0:
        return 0
    elif q == 0 or q == 1:
        return np.inf
    else:
        return p*np.log(p/q) + (1-p)*np.log((1-p)/(1-q))

In [ ]:
def dKL(p, q): # derivative of KL wrt q, p is constant
    result = (p-q)/(q*(q - 1.0))
    return result

In [ ]:
def newton(N, S, k, t, precision = 1e-3, max_iterations = 50, epsilon=1e-12):
    ''' Calculate upper confidence bound via newton's method 
    
        WARNING: This function works in that it efficiently finds greatest approx zero to f in (0,1).
        However, KL-UCB technically specifies that the returned value of q should be such that 
        f(q) <= 0. The q's returned by this function seemingly always satisfy f(q)>=0.
        Enforcing f(q) <= 0 does not work (times out) because f(q) converges to 0 from the right.
        If this is unacceptable, maybe look into other root finding methods like bisection.
    ''' 
    
    p = S[k]/N[k] # from paper
    delta = 0.1 # arbitrary small positive offset
    q = p + delta # initial guess. if p==q, then dKL=0 and we never move anywhere
    converged = False
    
    for n in range(max_iterations):
        if (p / q <= 0 or (1-p)/(1-q) <= 0): # sanity check for log domain errors
            print(f'log error: p={p}, q={q}, n={n}')
        # wish to find greatest val of q in (0,1) that gets f closest to zero from below
        f = KL(p, q) - np.log(t)/N[k]
        df = dKL(p, q) # derivative of f is just derivative of KL
        
        if abs(df) < epsilon: # check denominator not too small
            break 
        
        # NOTE: graph KL function to see that largest zero (what we want) is >= p.
        qnew = max(p+delta, min(q - (f / df), 1-epsilon)) # chris: my approach for keeping q in (p,1)
        print(f'q={q}, f(q)={f}, qnew={qnew}, precision={precision} n={n}')
        if(abs(f) < precision and abs(qnew - q) < precision): # check for early convergence
            converged = True
            print(f'converged at {n} iterations')
            break
        q = qnew
        
    if(converged == False):
        print("Did not converge")

    return q

In [ ]:
def KL_UCB(n, K, rwd_means):
    N = np.zeros(K)
    S = np.zeros(K)
    for t in range(K):
        N[t] = 1
        S[t] = rng.uniform(rwd_means[t]-.1, rwd_means[t]+.1) # Use uniform distribution for simplicity
    for t in range(K,n):
        a = np.argmax([newton(N, S, arm, t) for arm in range(K)]) #argmax part of line 6 of algorithm 1
        r = rng.uniform(rwd_means[a]-.1, rwd_means[a]+.1)
        N[a] = N[a] + 1
        S[a] = S[a] + r
    return N,S

In [ ]:
K = 5
rwd_means = [.2, .3, .4, .5, .6]
nums, rwds = KL_UCB(1000, 5, rwd_means)

In [ ]:
print(nums)

In [ ]:
print(rwds)